In [ ]:


import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications import ResNet50
import os
print(os.listdir("../input"))
sns.set(style='white', context='notebook', palette='deep')



In [ ]:
#loading data
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')


In [ ]:
Y_train = train_data['label']
sns.countplot(Y_train)
X_train = train_data.drop('label',axis = 1)
del train_data

In [ ]:
X_train.isnull().any().describe()

In [ ]:
test_data.isnull().any().describe()

In [ ]:
#Normalisation

X_train = X_train/255.0
test_data = test_data/255.0
type(X_train)

In [ ]:
#reshape
X_train = X_train.values.reshape(-1,28,28,1)
test_data = test_data.values.reshape(-1,28,28,1)
type(X_train)

In [ ]:
#label
Y_train = to_categorical(Y_train,num_classes = 10)

In [ ]:
#spliting training and testing data
random_seed = 2
#X_train,test_X,Y_train,test_Y = train_test_split(X_train,Y_train,test_size = 0.1,random_state = random_seed)
type(X_train)

In [ ]:
g = plt.imshow(X_train[0][:,:,0])

In [ ]:
generator = ImageDataGenerator(rescale = 1./255, 
                               rotation_range = 15, 
                               width_shift_range = 0.1, 
                               height_shift_range = 0.1,
                               shear_range = 0.1,
                               zoom_range = 0.1)

In [ ]:
myDataGen = generator.flow(X_train,Y_train, batch_size = 100)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), input_shape = (28,28,1), activation = 'relu', name = 'Conv2D1'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, (3,3), activation = 'relu', name = 'Conv2D2'))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128, (2,2),activation = 'relu', name = 'Conv2D3'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [ ]:

model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])


In [ ]:
hist  = model.fit_generator(myDataGen, steps_per_epoch = 350, epochs = 20)

In [ ]:
plt.plot(hist.history['acc'])

In [ ]:
result = model.predict(test_data)

In [ ]:
results = np.argmax(result,axis = 1)
results

In [ ]:
Label = pd.Series(results,name = 'Label')
ImageId = pd.Series(range(1,28001),name = 'ImageId')
submission = pd.concat([ImageId,Label],axis = 1)
submission.to_csv('submission.csv',index = False)